In [1]:
# Installing required packages
!pip install pyspark
!pip install findspark
import findspark
findspark.init()

# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context. 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Python Spark DataFrames basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
file = "train.csv"
df_train = spark.read.csv(file,header = True,inferSchema=True)

In [3]:
df_train.show(3)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 3 rows



## select 

In [5]:
df_train.select('PassengerId').show(10)

+-----------+
|PassengerId|
+-----------+
|          1|
|          2|
|          3|
|          4|
|          5|
|          6|
|          7|
|          8|
|          9|
|         10|
+-----------+
only showing top 10 rows



In [6]:
df_train.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In [12]:
file = "train.csv"
df_train = spark.read.csv(file,header = True,inferSchema=True)
df_train = df_train.drop('PassengerId','Ticket','Name','Cabin')

In [13]:
df_train.show(4)

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
|       1|     3|female|26.0|    0|    0|  7.925|       S|
|       1|     1|female|35.0|    1|    0|   53.1|       S|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 4 rows



## select multiple columns

In [14]:
df_train.select('Sex','Pclass').show(3)

+------+------+
|   Sex|Pclass|
+------+------+
|  male|     3|
|female|     1|
|female|     3|
+------+------+
only showing top 3 rows



## distinct

In [16]:
df_train.select('Sex').distinct().show()

+------+
|   Sex|
+------+
|female|
|  male|
+------+



In [19]:
df_train.select('Pclass').distinct().show()

+------+
|Pclass|
+------+
|     1|
|     3|
|     2|
+------+



In [22]:
for i in df_train.columns:
    print(i , df_train.select(i).distinct().count())
    print(df_train.select(i).distinct().show())

Survived 2
+--------+
|Survived|
+--------+
|       1|
|       0|
+--------+

None
Pclass 3
+------+
|Pclass|
+------+
|     1|
|     3|
|     2|
+------+

None
Sex 2
+------+
|   Sex|
+------+
|female|
|  male|
+------+

None
Age 89
+----+
| Age|
+----+
| 8.0|
|70.0|
| 7.0|
|20.5|
|49.0|
|29.0|
|40.5|
|64.0|
|47.0|
|42.0|
|24.5|
|44.0|
|35.0|
|null|
|62.0|
|18.0|
|80.0|
|34.5|
|39.0|
| 1.0|
+----+
only showing top 20 rows

None
SibSp 7
+-----+
|SibSp|
+-----+
|    1|
|    3|
|    5|
|    4|
|    8|
|    2|
|    0|
+-----+

None
Parch 7
+-----+
|Parch|
+-----+
|    1|
|    6|
|    3|
|    5|
|    4|
|    2|
|    0|
+-----+

None
Fare 248
+-------+
|   Fare|
+-------+
|   15.5|
| 133.65|
| 8.5167|
| 29.125|
|10.4625|
| 7.0458|
|  9.475|
|11.1333|
|    0.0|
| 7.7333|
|   73.5|
|77.2875|
|   15.9|
|   11.5|
| 8.6833|
|41.5792|
|    9.5|
| 8.4042|
|14.4542|
|14.4583|
+-------+
only showing top 20 rows

None
Embarked 4
+--------+
|Embarked|
+--------+
|       Q|
|    null|
|       C|
|     

In [26]:
from pyspark.sql.functions import col, countDistinct

df_train.agg(*(countDistinct(col(c)).alias(c) for c in df_train.columns)).show()

+--------+------+---+---+-----+-----+----+--------+
|Survived|Pclass|Sex|Age|SibSp|Parch|Fare|Embarked|
+--------+------+---+---+-----+-----+----+--------+
|       2|     3|  2| 88|    7|    7| 248|       3|
+--------+------+---+---+-----+-----+----+--------+



In [32]:
df_train.select(countDistinct(col("Sex"))).show()

+-------------------+
|count(DISTINCT Sex)|
+-------------------+
|                  2|
+-------------------+



In [33]:
df_train.select('Sex').distinct().count()

2

## value_counts

In [42]:
df_train.groupBy('Pclass').count().orderBy('Pclass').show()

+------+-----+
|Pclass|count|
+------+-----+
|     1|  216|
|     2|  184|
|     3|  491|
+------+-----+



In [44]:
df_train.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)



In [55]:
from pyspark.sql.functions import col,isnan,when,count
df_train.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_train.columns]
   ).show()

+--------+------+---+---+-----+-----+----+--------+
|Survived|Pclass|Sex|Age|SibSp|Parch|Fare|Embarked|
+--------+------+---+---+-----+-----+----+--------+
|       0|     0|  0|177|    0|    0|   0|       2|
+--------+------+---+---+-----+-----+----+--------+



In [65]:
from pyspark.sql.functions import mean 

df_train.select(*(mean(col(c)).alias(c)  for c in df_train.columns)).show()

+------------------+-----------------+----+-----------------+------------------+-------------------+----------------+--------+
|          Survived|           Pclass| Sex|              Age|             SibSp|              Parch|            Fare|Embarked|
+------------------+-----------------+----+-----------------+------------------+-------------------+----------------+--------+
|0.3838383838383838|2.308641975308642|null|29.69911764705882|0.5230078563411896|0.38159371492704824|32.2042079685746|    null|
+------------------+-----------------+----+-----------------+------------------+-------------------+----------------+--------+



In [68]:
df_train_summ= df_train.describe()

In [72]:
df_train_summ.select('Summary','Pclass').show()

+-------+------------------+
|Summary|            Pclass|
+-------+------------------+
|  count|               891|
|   mean| 2.308641975308642|
| stddev|0.8360712409770491|
|    min|                 1|
|    max|                 3|
+-------+------------------+



In [73]:
df_train_summ.select('Summary','Age').show()

+-------+------------------+
|Summary|               Age|
+-------+------------------+
|  count|               714|
|   mean| 29.69911764705882|
| stddev|14.526497332334035|
|    min|              0.42|
|    max|              80.0|
+-------+------------------+



In [77]:
df_train.dtypes

[('Survived', 'int'),
 ('Pclass', 'int'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Fare', 'double'),
 ('Embarked', 'string')]